In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_path = "../data/BOLT Data Set.xlsx"
og_df = pd.read_excel(data_path)
og_df.head()

,Card Identifier,Transaction Date,Transaction Time,Risk Assessment,Payment Method,Transaction Value,Merchant Location,Card Present Status,Chip Usage,Cross-border Transaction (Yes/No),Acquiring Institution ID,Merchant Identifier,Merchant Category Code (MCC),Fraud Indicator (Yes/No)
0,card 1,2023-05-11,16:22:14.0,362.0,Paypass - Contactless,13.98,USA,CP,Yes,No,acquirer 1,merchant 1,5812.0,No
1,card 2,2023-06-05,15:16:35.0,602.0,Online,24.64,USA,CNP,No,No,acquirer 2,merchant 2,4121.0,No
2,card 3,2023-06-05,11:57:40.0,482.0,Unknown,15.00,USA,CNP,No,No,acquirer 3,merchant 3,7211.0,No
3,card 4,2023-07-20,18:26:30.0,947.0,Online,30.56,USA,CNP,No,No,acquirer 1,merchant 4,5814.0,No
4,card 5,2023-03-03,19:03:11.0,1382.0,Magnetic Stripe,50.85,USA,CP,No,No,acquirer 4,merchant 1,5812.0,No


In [3]:
df = og_df.copy()

In [4]:
from datetime import datetime
def fix_time(time_str):
    og_time = time_str
    if time_str == "na:n:":
        return np.nan
    try:
        time_str = time_str.replace(":", "").replace(".0", "")
        time_str = '{:0>{}}'.format(time_str, 6)
        time_fixed = datetime.strptime(time_str, '%H%M%S')#.strftime('%H:%M:%S')
        return time_fixed
    except Exception as e:
        print(e)
        print(og_time)
        return np.nan

df['Transaction Time'] = df['Transaction Time'].apply(fix_time)


In [10]:
fraud = df[df['Fraud Indicator (Yes/No)'] == 'Yes']
fraud_df = df[df['Card Identifier'].isin(fraud['Card Identifier'])]

In [26]:
fraud.info()

<class 'pandas.core.frame.DataFrame'>
Index: 265 entries, 41 to 99471
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Card Identifier                    265 non-null    object        
 1   Transaction Date                   265 non-null    datetime64[ns]
 2   Transaction Time                   264 non-null    datetime64[ns]
 3   Risk Assessment                    265 non-null    float64       
 4   Payment Method                     265 non-null    object        
 5   Transaction Value                  265 non-null    float64       
 6   Merchant Location                  265 non-null    object        
 7   Card Present Status                265 non-null    object        
 8   Chip Usage                         265 non-null    object        
 9   Cross-border Transaction (Yes/No)  265 non-null    object        
 10  Acquiring Institution ID           265 n

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 14 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   Card Identifier                    100000 non-null  object        
 1   Transaction Date                   100000 non-null  datetime64[ns]
 2   Transaction Time                   97963 non-null   datetime64[ns]
 3   Risk Assessment                    99028 non-null   float64       
 4   Payment Method                     100000 non-null  object        
 5   Transaction Value                  100000 non-null  float64       
 6   Merchant Location                  100000 non-null  object        
 7   Card Present Status                100000 non-null  object        
 8   Chip Usage                         100000 non-null  object        
 9   Cross-border Transaction (Yes/No)  100000 non-null  object        
 10  Acquiring Institution

In [5]:
df.head()

,Card Identifier,Transaction Date,Transaction Time,Risk Assessment,Payment Method,Transaction Value,Merchant Location,Card Present Status,Chip Usage,Cross-border Transaction (Yes/No),Acquiring Institution ID,Merchant Identifier,Merchant Category Code (MCC),Fraud Indicator (Yes/No)
0,card 1,2023-05-11,1900-01-01 16:22:14,362.0,Paypass - Contactless,13.98,USA,CP,Yes,No,acquirer 1,merchant 1,5812.0,No
1,card 2,2023-06-05,1900-01-01 15:16:35,602.0,Online,24.64,USA,CNP,No,No,acquirer 2,merchant 2,4121.0,No
2,card 3,2023-06-05,1900-01-01 11:57:40,482.0,Unknown,15.00,USA,CNP,No,No,acquirer 3,merchant 3,7211.0,No
3,card 4,2023-07-20,1900-01-01 18:26:30,947.0,Online,30.56,USA,CNP,No,No,acquirer 1,merchant 4,5814.0,No
4,card 5,2023-03-03,1900-01-01 19:03:11,1382.0,Magnetic Stripe,50.85,USA,CP,No,No,acquirer 4,merchant 1,5812.0,No


In [16]:
183 / 7872

0.023246951219512195

In [15]:
fraud_df['Card Identifier'].nunique()

183

In [11]:
df['Card Identifier'].nunique()

7872

In [12]:
df['Merchant Identifier'].nunique()

2658

In [8]:
df.shape

(100000, 14)

## Preprocessing
- Scaling on numeric features: risk assessment, transaction value
- One hot encoding on categorical features: payment method, merchant location, card present status, chip usage, cross-border transaction, MCC

## Linear Regression on Risk Assessment

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [28]:
lr_df = df.dropna(subset=['Risk Assessment'])

In [29]:
lr_df.shape

(99028, 14)

In [21]:
# Separate features and target variable
X = df[['Risk Assessment']]  # Feature: Risk Assessment
y = df['Fraud Indicator (Yes/No)'].map({'Yes': 1, 'No': 0})    # Target variable to predict

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Separate features and target variable
X = lr_df[['Risk Assessment']]  # Feature: Risk Assessment
y = lr_df['Fraud Indicator (Yes/No)'].map({'Yes': 1, 'No': 0})    # Target variable to predict

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize logistic regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)


Accuracy: 0.9971725739674846
Confusion Matrix:
 [[19750     0]
 [   56     0]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     19750
           1       0.00      0.00      0.00        56

    accuracy                           1.00     19806
   macro avg       0.50      0.50      0.50     19806
weighted avg       0.99      1.00      1.00     19806



C:\Users\simon\anaconda3\envs\cpsc330\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\simon\anaconda3\envs\cpsc330\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\simon\anaconda3\envs\cpsc330\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
